In [ ]:
# Load libraries

import numpy as np # linear algebra
import pandas as pd # data processing
# from sklearn.preprocessing import Imputer --> doesn't work
from sklearn.impute import SimpleImputer 
from sklearn.neighbors import NearestNeighbors 
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# 1. Load data

def initialise_data(file_name, cols=[]):

    """
    This function loads the file into a dataframe and imports the packages needed, ready for cleaning.

    Parameters:
    file_name : specify the directory path and file name of the dataset
    file_format :  provide the format of the original tabular dataset (eg., csv, excel) 
    columns : provide the name of the columns as a list for the dataframe, ensure list length = number of columns
    """
    
    x = {"xlsx": pd.read_excel, "csv": pd.read_csv} 
    df = x[file_name.split(".")[1]](file_name, header=None, index_col=False)
    df.columns = cols
    return df

cols = [
    'Unique Meter ID', 
    'Unix Time Stamp', 
    'Date/Time Stamp', 
    'Incremental Consumption Value (Gallons)', 
    'Reading Value (Gallons)',
    'Some data'
]

df = initialise_data('Sample_UtilityX_AMIData.csv',cols)
df.head()

In [ ]:
# 2. Converting to date-time 

def convert_to_datetime(df, time_col):
    """
    Functions convert time column to datetime format
    df: input your dataframe
    time_col: the time column in your dataframe
    """
    df[time_col] = pd.to_datetime(df[time_col])
    return df

df = convert_to_datetime(df, 'Date/Time Stamp')

In [ ]:
# 3.1. Negative value check

# Increase efficiency 

def negative_value_check(df, opt, id_col, cons_col):
    """
    id_col is header of column with Meter ID
    cons_col is header of column with Consumption Values
    dataframe is your full dataframe
    opt is either 'remove' (removes meters with negative values), 
    'show' (shows ALL meter entries with negative values),
    'list' (lists UNIQUE meter IDs with negative values in an array) or 'both' 
    (returns a new dataframe with both negative and positive values and marks them)
    """

    df['Negative?'] = 0
    df.loc[df[cons_col] < 0, 'Negative?'] = 1

    if opt == 'remove':
        df = df.loc[df['Negative?'] == 0]
        df.drop('Negative?', axis=1, inplace=True)
        return df
    elif opt == 'show':
        df = df.loc[df['Negative?'] == 1]
        df.drop('Negative?', axis=1, inplace=True)
        return df
    elif opt == 'list':
        df = df.loc[df[cons_col] < 0, id_col].unique()
        df.drop('Negative?', axis=1, inplace=True)
        return df
    
negative_value_check(df, 'show','Unique Meter ID','Incremental Consumption Value (Gallons)')

In [ ]:
# 3.2. NaN value check

def NaN_value_check(df, opt, id_col, cons_col):
    """
    dataframe is your full dataframe
    id_col is header of column with Meter ID
    cons_col is header of column with Consumption Values
    opt is either 'remove' (removes meters with negative values), 
    'show' (shows ALL meter entries with negative values),
    'list' (lists UNIQUE meter IDs with negative values in an array) or 'both' 
    (returns a new dataframe with both negative and positive values and marks them)
    """
    
    if opt == 'remove':
        return df.loc[df[id_col].isna()]
    elif opt == 'show':
        return df.loc[df[id_col].isna() == True]
    elif opt == 'list':
        pass # complete 
    return df

NaN_value_check(df, 'show', 'Unique Meter ID','Incremental Consumption Value (Gallons)')

In [ ]:
df.head()

In [ ]:
df['Incremental Consumption Value (Gallons)'].isna() 

# df[df['Incremental Consumption Value (Gallons)'].isna() == False]

In [ ]:
df.drop[df['Incremental Consumption Value (Gallons)'] == np.NaN]

In [ ]:
[df['Incremental Consumption Value (Gallons)'] == np.NaN]

In [ ]:
df.loc[df['Reading Value (Gallons)'].isna() == True]

In [ ]:
df.loc[df['Some data'].isna() == True]

In [ ]:
type(random)

In [ ]:
df['Negative?'] = 0
df.loc[df['Incremental Consumption Value (Gallons)'] < 0, 'Negative?'] = 1
df.loc[df['Negative?'] == 1]

# Weird results
# Increment value says -10, yet Reading value increases by 10
# Also timestamps are not chronological
# Some timestamps are missing

In [ ]:
# 4. Visualise gaps in data
# make time-related columns for analysis over various periods
df['dotw'] = df['Date/Time Stamp'].dt.dayofweek
df['hour'] = df['Date/Time Stamp'].dt.hour
df['doty'] = df['Date/Time Stamp'].dt.dayofyear

In [ ]:
# 4.1. Diurnal cycles

sns.lineplot(
    x='hour',
    y='Incremental Consumption Value (Gallons)',
    data=df
)
# Line indicates mean each hour
# Shadow indicates 95% confidence interval
# Plot correponds to expected behaviour, high in morning, late afternoon and evening/night

In [ ]:
# 4.2. Weekly cycles 

sns.lineplot(
    x='dotw',
    y='Incremental Consumption Value (Gallons)',
    data=df
)

# Strange behaviour, should be consistent through weekdays 
# Delve deeper into data

In [ ]:
# 4.3. Yearly cycles

sns.lineplot(
    x='doty',
    y='Incremental Consumption Value (Gallons)',
    data=df
)

# This definitely deviates from expectations!
# Huge dips in spring/summer
# Random dips occassionally

In [ ]:
df = df.join(
    df['Unique Meter ID'].value_counts(), 
    on='Unique Meter ID',
    rsuffix='000',
    how='left'
)

In [ ]:
df.columns = ['Unique Meter ID', 'Unix Time Stamp', 'Date/Time Stamp',
       'Incremental Consumption Value (Gallons)', 'Reading Value (Gallons)',
       'Negative?','dotw', 'hour', 'doty' ,'data_count']

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,10))
sns.lineplot(
    x='doty',
    y='Incremental Consumption Value (Gallons)',
    data=df.loc[df['data_count'] > 8700],
    ax=ax
)

In [ ]:
data = df.loc[df['data_count'] > 8700]
data.describe()

In [ ]:
daily_cons = raw_df.groupby(['Unique Meter ID', 'doty']).\
    agg({'Incremental Consumption Value (Gallons)':'sum'})

In [ ]:
# fill data set missing value with mean
def fill_with_mean(data, features, show_info=False):
    """
    :type data: pandas DataFrame
    """
    df = data.loc[:, features]
    print('begin filling...')
    for ele in features:
        if show_info == True:
            print(ele)
        p = df.loc[:, ele]
        p[p.isna()] = p[p.notna()].mean()
        df[ele] = p
    print('finished! merge result...')
    for ele in features:
        if show_info == True:
            print(ele)
        data[ele] = df.loc[:, ele]
    print('successflly done!')
    return data

In [ ]:
# fill data set missing value with mean (integer mean)
def fill_with_int_mean(data, features, show_info=False):
    """
    :type data: pandas DataFrame
    """
    df = data.loc[:, features]
    print('begin filling...')
    for ele in features:
        if show_info == True:
            print(ele)
        p = df.loc[:, ele]
        p[p.isna()] = int(p[p.notna()].mean())
        df[ele] = p
    print('finished! merge result...')
    for ele in features:
        if show_info == True:
            print(ele)
        data[ele] = df.loc[:, ele]
    print('successflly done!')
    return data


In [ ]:
# fill datas et missing value with most frequent value
def fill_with_mode(data, features, show_info=False):
    """
    :type data: pandas DataFrame
    """
    df = data.loc[:, features]
    print('begin filling...')
    for ele in features:
        if show_info == True:
            print(ele)
        p = df.loc[:, ele]
        p[p.isna()] = p[p.notna()].mode()[0]
        df[ele] = p
    print('finished! merge result...')
    for ele in features:
        if show_info == True:
            print(ele)
        data[ele] = df.loc[:, ele]
    print('successflly done!')
    return data

In [ ]:
# fill data set missing value with median
def fill_with_median(data):
    imp = Imputer(missing_values='NaN', strategy='median', axis=0)
    if len(data.shape) == 1:
        data_new = imp.fit_transform(data.values.reshape(-1, 1))
    else:
        data_new = imp.fit_transform(data.values)
    data.values = data_new
    return data

In [ ]:
# 5. Fill Missing values
# Using kNN to predict missing value
def fill_with_KNN_(data, best_k, train_feature_list, target_feature, model='class'):
    predictor = data[train_feature_list]
    target = data[target_feature]
    index_na = target[target.isna()].index.values
    index_notna = target[target.notna()].index.values

    X_train = predictor.loc[index_notna, :].values
    y_train = target.loc[index_notna].values
    X_test = predictor.loc[index_na, :].values

    print('begin train ......')
    if model == 'class':
        knn = KNeighborsClassifier(n_neighbors=best_k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
    if model == 'reg':
        knn = KNeighborsRegressor(n_neighbors=best_k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
    print('successful preidct')

    p1 = target.loc[index_na]
    p2 = pd.Series(y_pred, index=p1.index.values)
    p3 = target[target.notna()]
    p2 = p2.append(p3)
    p2 = p2.sort_index(axis=0)
    data[target_feature] = p2
    print('successfully deal with missing value with data.')

    return data

In [ ]:
# 6. Interpolate missing values
